# Clickstream Clustering using Weighted Longest Common Subsequences

This is scientific article implementation: Arindam Banerjee and Joydeep Ghosh, _Clickstream Clustering Using Weighted Longest Common Subsequences_, (2001) [link here](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.135.9092).

The implementation is divided into 3 parts:
* First, paths similarity measure is calculated for all paths
* Second, similarity graph is constructed
* Third, graph partitioning algorithm is executed

As it was impossible to get the same as in article data from [www.sulekha.com](https://www.sulekha.com), the weblogs from [www.bodyworlds.nl](https://www.bodyworlds.nl/nl) were considered instead. This choice is temporal, and data from e.g. NASA Kennedy Space Center [www.ita.ee.lbl.gov](http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html) should be used instead as those are the classical choice for web mining applications. The [www.bodyworlds.nl](https://www.bodyworlds.nl/nl) was chosen as these weblogs require minimum initial preprocessing.

In [22]:
import math

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from statistics import mean 

In [ ]:
# data format - list of tuples
# [('path1', time1), ('path2', time2), ('path3', time3), ...]

In [3]:
test_data = [
    {'session_id': 1, 
     'page_views': [('website', 20), ('tickets', 10), ('convious', 2), ('choose-your-tickets', 10), ('cart', 30), ('order', 1), ('get-your-tickets', 3)]},
    {'session_id': 2, 
     'page_views': [('website', 5), ('about', 30)]},
    {'session_id': 3, 
     'page_views': [('website', 45), ('tickets', 45), ('convious', 34), ('choose-your-tickets', 55)]},
    {'session_id': 4, 
     'page_views': [('website', 23), ('about', 45), ('tickets', 23), ('contact', 21), ('convious', 4)]}
]
test_df = pd.DataFrame(test_data)

In [4]:
test_df.head()

,page_views,session_id
0,"[(website, 20), (tickets, 10), (convious, 2), ...",1
1,"[(website, 5), (about, 30)]",2
2,"[(website, 45), (tickets, 45), (convious, 34),...",3
3,"[(website, 23), (about, 45), (tickets, 23), (c...",4


## Paths similarity

In [66]:
# source: https://github.com/man1/Python-LCS
# get the matrix of LCS lengths at each sub-step of the recursive process
# (m+1 by n+1, where m=len(list1) & n=len(list2) ... it's one larger in each direction 
# so we don't have to special-case the x-1 cases at the first elements of the iteration
def lcs_mat(list1, list2):
    m = len(list1)
    n = len(list2)
    # construct the matrix, of all zeroes
    mat = [[0] * (n+1) for row in range(m+1)]
    # populate the matrix, iteratively
    for row in range(1, m+1):
        for col in range(1, n+1):
            if list1[row - 1] == list2[col - 1]:
                # if it's the same element, it's one longer than the LCS of the truncated lists
                mat[row][col] = mat[row - 1][col - 1] + 1
            else:
                # they're not the same, so it's the the maximum of the lengths of the LCSs of the two options (different list truncated in each case)
                mat[row][col] = max(mat[row][col - 1], mat[row - 1][col])
    # the matrix is complete
    return mat

# backtracks all the LCSs through a provided matrix
def all_lcs(lcs_dict, mat, list1, list2, index1, index2):
    #calculate recursively
    if (index1 == 0) or (index2 == 0): # base case
        return [[]]
    elif list1[index1 - 1] == list2[index2 - 1]:
        # elements are equal! Add it to all LCSs that pass through these indices
        lcs_dict[(index1, index2)] = [prevs + [list1[index1 - 1]] for prevs in all_lcs(lcs_dict, mat, list1, list2, index1 - 1, index2 - 1)]
        return lcs_dict[(index1, index2)]
    else:
        lcs_list = [] # set of sets of LCSs from here
        # not the same, so follow longer path recursively
        if mat[index1][index2 - 1] >= mat[index1 - 1][index2]:
            before = all_lcs(lcs_dict, mat, list1, list2, index1, index2 - 1)
            for series in before: # iterate through all those before
                if not series in lcs_list: lcs_list.append(series) # and if it's not already been found, append to lcs_list
        if mat[index1 - 1][index2] >= mat[index1][index2 - 1]:
            before = all_lcs(lcs_dict, mat, list1, list2, index1 - 1, index2)
            for series in before:
                if not series in lcs_list: lcs_list.append(series)
        lcs_dict[(index1, index2)] = lcs_list
        return lcs_list

# return a set of the sets of longest common subsequences in list1 and list2
def lcs(list1, list2):
    # mapping of indices to list of LCSs, so we can cut down recursive calls enormously
    mapping = dict()
    return all_lcs(mapping, lcs_mat(list1, list2), list1, list2, len(list1), len(list2))

In [65]:
# return filtered subsequences with time
def time_lcs(subsequence, list_of_dict):
    # TODO: decide what to do if several subsequences of the same length are found
    subsequence = subsequence[0]
    return [t for t in list_of_dict if t[0] in subsequence]

In [64]:
# compute composed similarity measure
def similarity(list_of_paths1, list_of_paths2):
    paths1 = [seq[0] for seq in list_of_paths1]
    T1 = sum([seq[1] for seq in list_of_paths1])
    
    paths2 = [seq[0] for seq in list_of_paths2]
    T2 = sum([seq[1] for seq in list_of_paths2])
    
    common_subsequence = lcs(paths1, paths2)
    path_subsequence1 = time_lcs(common_subsequence, list_of_paths1)
    t1 = sum([seq[1] for seq in path_subsequence1])
    path_subsequence2 = time_lcs(common_subsequence, list_of_paths2)
    t2 = sum([seq[1] for seq in path_subsequence2])
    
    return s_similarity(path_subsequence1, path_subsequence2) * s_importance(t1, T1, t2, T2)

# compute similarity component
def s_similarity(t_alpha, t_beta):
    s = [min(t_a[1], t_b[1])/max(t_a[1], t_b[1]) for t_a, t_b in zip(t_alpha, t_beta)]
    return mean(s)

# compute importance component
def s_importance(t_alpha, T_alpha, t_beta, T_beta):
    return math.sqrt(t1/T1 * t2/T2)

## Similarity graph

In [88]:
def similarity_graph(df, edge_threshold=.1):
    # create graph object
    g = nx.Graph()
    
    # add nodes
    g.add_nodes_from(list(df.session_id))
    
    # add egdes
    for session_1 in range(len(df)):
        for session_2 in range(session_1 + 1, len(df)):
            edge_weight = similarity(df.iloc[session_1].page_views, df.iloc[session_2].page_views)
            if edge_weight >= edge_threshold:
                g.add_edge(df.iloc[session_1].session_id, df.iloc[session_2].session_id, weight=edge_weight)
    return g

In [89]:
G = similarity_graph(test_df)

## Graph partitioning

In [ ]:
# metis with min-cut and balancing constraints